In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import json

from matplotlib import pyplot as plt
import pystruct as pystr

%matplotlib inline

# Load data

In [2]:
with open('slotfilling-data.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

In [3]:
def process_data(data):
    X = [item['chat'] for item in data]
    y = []
    for item in data:
        entities = item['entities']
        y_item = {}
        for entity in entities:
            y_item[entity['title']] = {
                'start_pos': entity['start_pos'],
                'end_pos': entity['end_pos'],
                'text': entity['text']
            }

        y.append(y_item)
    
    return np.array(X), np.array(y)

In [4]:
data, ans = process_data(data)
len(data)

6500

In [5]:
# Mix the data
perm = np.random.permutation(len(data))
data, ans = data[perm], ans[perm]

In [6]:
possible_slots = set([item for y_item in ans for item in list(y_item.keys())])
possible_slots

{'ВАЛЮТА',
 'ВРЕМЯ_ДАТА_СНЯТИЯ',
 'ЗА_ГРАНИЦЕЙ',
 'МЕСТО_СНЯТИЯ',
 'НАЗВАНИЕ_БАНКА',
 'НОМЕР_ТЕЛЕФОНА',
 'РАЗМЕР_КОМИССИИ',
 'СУММА_СНЯТИЯ',
 'ТАРИФ_КАРТЫ',
 'ТИП_КАРТЫ'}

In [7]:
(data[0], ans[0])

('1: Добрый день! Скажите, какие условия на снятие наличных в банкоматах ща границей? Карта Black\n2: Снятие в любых банкоматах без комиссии при выполнении условий тарифного плана. Однако, некоторые банки могут взимать собственную комиссия за свои услуги. Об этом информация отражается непосредственно перед снятием.\n1: В условиях тарифа я нашел информацию только по тарифам в рублях.\n1: Какая сумма эквивалентна 1111 р в, предполодим, долларах сша?\n1: по курсу цб рф? или это 111$, как было раньше?\n2: Сумма должна быть больше 1111 руб. с учетом конвертации. Ориентироваться нужно на курс Тинькофф Банка.\n2: https://www.tinkoff.ru/about/documents/exchange/\n2: "Для дебетовых карт" далее "С использованием ее реквизитов".\n',
 {'ВАЛЮТА': {'end_pos': 478, 'start_pos': 477, 'text': '$'},
  'СУММА_СНЯТИЯ': {'end_pos': 477, 'start_pos': 474, 'text': '111'},
  'ТАРИФ_КАРТЫ': {'end_pos': 96, 'start_pos': 89, 'text': 'Black'}})

# Vectorize data

In [8]:
from gensim.models import Word2Vec

In [9]:
model = Word2Vec.load('word2vec/w2v_model_tfidf_size300_window5_mc2.w2v')

In [10]:
import re
# Список списокв извлеченных из текстов слов
wordList =  [[x.lower() for x in re.findall(r"[\w']+", y)] for y in data]

In [11]:
# Объединение всех слов из выборки в один уникальный список
unique_words = list(set([item for sublist in wordList for item in sublist]))

In [12]:
# Формирование списка векторов данных из word2vec
X = []
for dialog in data:
    line = []
    for word in dialog:
        try:
            line.append(model[word])
        except:
            pass
    X.append(line)

In [49]:
# Формирование списка векторов ответов из word2vec
y = {'ВАЛЮТА': [], 'ВРЕМЯ_ДАТА_СНЯТИЯ': [], 'ЗА_ГРАНИЦЕЙ': [], 'МЕСТО_СНЯТИЯ': [],
 'НАЗВАНИЕ_БАНКА': [], 'НОМЕР_ТЕЛЕФОНА': [], 'РАЗМЕР_КОМИССИИ': [],
 'СУММА_СНЯТИЯ': [], 'ТАРИФ_КАРТЫ': [], 'ТИП_КАРТЫ': []}
n = len(model['тиньков'])
for line in ans:
    for slot in possible_slots:
        try:
            y[slot].append(model[line[slot]['text']])
            #print(line[slot]['text'])
        except:
            y[slot].append(np.zeros(n))
            

In [14]:
# Обратное преобразование
word=model.most_similar(positive=[model['тиньков']],topn=1)
print(word[0][0])

тиньков


# Solution

In [15]:
X_train, y_train, X_test, y_test = data[:6000], ans[:6000], data[6000:], ans[6000:]

In [16]:
class SimpleSolutionModel:
    def __init__(self):
        self._text_to_slot = {}
    
    def fit(self, X, y):
        for y_item in y:
            for slot_title, slot_info in y_item.items():
                self._text_to_slot[slot_info['text']] = slot_title
            
    def predict(self, X):
        y = []
        
        for x_item in X:
            y_item = {}
            for slot_text, slot_title in self._text_to_slot.items():
                index = x_item.find(slot_text)
                if index != -1:
                    y_item[slot_title] = { 
                        'start_pos': index, 
                        'end_pos': index + len(slot_text), 
                        'text': slot_text
                    }
                
            y.append(y_item)
            
        return y

In [17]:
model = SimpleSolutionModel()

model.fit(X_train, y_train)

In [18]:
y_pred = model.predict(X_test)

# Evaluation

In [19]:
def tokenize(token):
    return ''.join([char for char in token if char not in ['.']])

def q_distance(tokens_test, tokens_pred):
    tokens_test = [tokenize(token) for token in tokens_test]
    tokens_pred = [tokenize(token) for token in tokens_pred]
    
    common = len(set(tokens_test) & set(tokens_pred))
    fp = len(set(tokens_pred) - set(tokens_test))
    fn = len(set(tokens_test) - set(tokens_pred))
    
    return common / (common + fp + fn)

def precision_on_dataset(X, y, y_pred):
    """
    X_test - array of chats
    y_test - hash with slots { 'SLOT_NAME': { 'start_pos': 123, 'end_pos': 135 }, ... }
    y_pred - hash_with_predicted_slots
    """
    
    q_sum = 0
    total = 0
    
    for x_item, y_item, y_pred_item in tqdm(zip(X, y, y_pred)):
        for slot_title, y_pred_slot_info in y_pred_item.items():
            if slot_title in y_item:
                y_pred_tokens = x_item[y_pred_slot_info['start_pos']:y_pred_slot_info['end_pos']].split(' ')
                y_tokens = x_item[y_item[slot_title]['start_pos']:y_item[slot_title]['end_pos']].split(' ')
                
                q_sum += q_distance(y_tokens, y_pred_tokens)
            
            total += 1
            
    return q_sum / total

def recall_on_dataset(X, y, y_pred):
    """
    X_test - array of chats
    y_test - hash with slots { 'SLOT_NAME': { 'start_pos': 123, 'end_pos': 135 }, ... }
    y_pred - hash_with_predicted_slots
    """
    
    q_sum = 0
    total = 0
    
    for x_item, y_item, y_pred_item in tqdm(zip(X, y, y_pred)):
        for slot_title, y_pred_slot_info in y_item.items():
            if slot_title in y_pred_item:
                y_pred_tokens = x_item[y_pred_slot_info['start_pos']:y_pred_slot_info['end_pos']].split(' ')
                y_tokens = x_item[y_item[slot_title]['start_pos']:y_item[slot_title]['end_pos']].split(' ')
                
                q_sum += q_distance(y_tokens, y_pred_tokens)
            
            total += 1
            
    return q_sum / total

def f1_on_dataset(X, y, y_pred):
    precision = precision_on_dataset(X, y, y_pred)
    recall = recall_on_dataset(X, y, y_pred)
    
    return 2 * precision * recall / (precision + recall)

In [20]:
f1_on_dataset(X_test, y_test, y_pred)

500it [00:00, 14057.58it/s]
500it [00:00, 15976.96it/s]


0.15824120499541205

In [21]:
for i in range(len(ans)):
    print (data[i], ans[i])

1: Добрый день! Скажите, какие условия на снятие наличных в банкоматах ща границей? Карта Black
2: Снятие в любых банкоматах без комиссии при выполнении условий тарифного плана. Однако, некоторые банки могут взимать собственную комиссия за свои услуги. Об этом информация отражается непосредственно перед снятием.
1: В условиях тарифа я нашел информацию только по тарифам в рублях.
1: Какая сумма эквивалентна 1111 р в, предполодим, долларах сша?
1: по курсу цб рф? или это 111$, как было раньше?
2: Сумма должна быть больше 1111 руб. с учетом конвертации. Ориентироваться нужно на курс Тинькофф Банка.
2: https://www.tinkoff.ru/about/documents/exchange/
2: "Для дебетовых карт" далее "С использованием ее реквизитов".
 {'СУММА_СНЯТИЯ': {'start_pos': 474, 'end_pos': 477, 'text': '111'}, 'ТАРИФ_КАРТЫ': {'start_pos': 89, 'end_pos': 96, 'text': 'Black'}, 'ВАЛЮТА': {'start_pos': 477, 'end_pos': 478, 'text': '$'}}
1: Добрый день . Хотел бы уточнить у Вас , я могу снять деньги наличными со своей дебет

 {'ТИП_КАРТЫ': {'start_pos': 66, 'end_pos': 74, 'text': 'рублёвой'}, 'МЕСТО_СНЯТИЯ': {'start_pos': 108, 'end_pos': 116, 'text': 'Харьков'}, 'ВАЛЮТА': {'start_pos': 366, 'end_pos': 372, 'text': 'гривна'}}
1: Здравствуйте! Как мне узнать пин код своей карты? мне говорили по телефону, но я забыл
2: Вам будет удобно принять звонок от сотрудника банка в течение нескольких минут?
 {}
1: Здравствуйте! Я забыла пинкод и 1 раза ввела ошибочный код в банкомате. Карта была заблокирована. Как мне ее разблокировать?
2: Вы можете позвонить в банк по номеру 1 111 111 11 11 (звонок по России бесплатный), чтобы разблокировать ПИН-код.
1: я сейчас за границей, в роуминге. Нет ли возможности снять блок онлайн?
2: Сожалею, это возможно только в рамках звонка. Могу передать коллегам, чтобы с Вами связались, либо Вы можете воспользоваться онлайн-звонком на сайте.
 {}
1: Добрый день. Пыталась сейчас снять деньги в банкомате, оказалось, что не помню пин. Что делать?
2: Если Вы забыли Ваш ПИН-код, позвоните по

 {'ТИП_КАРТЫ': {'start_pos': 32, 'end_pos': 39, 'text': '$ карты'}, 'ВАЛЮТА': {'start_pos': 18, 'end_pos': 19, 'text': '$'}}
1: Добрый день! Я забыл свой Пин-код. Вышлите мне, пожалуйста, его.
2: Мы не храним информацию о ПИН-коде, восстановить его нельзя. Вы можете получить новый ПИН по карте, для этого нужно позвонить в банк. Или могу передать коллегам, чтобы самостоятельно связались с Вами.
1: Да, передайте, пожалуйста. Скоро будут покупки по расчётной карте. Жду звонка. Спасибо!
 {}
1: Здравствуйте! Потерял пин код от карты, можно ли его восстановить или заменить без перевыпуска карты?
2: Вам удобно будет сейчас принять звонок от сотрудника банка?
 {}
1: Здравствуй! Если я забыл пин от карты, как-нибудь его возможно восстановить?
2: Сможете принять звонок на контактный телефон для изменения ПИН-кода?
2: Нет, практически сразу должно быть подтверждение.
1: Ой, здравствуйте!
2: Сейчас посмотрю информацию по Вашей карте, 1 минуту.
1: Хорошо, жду
2: ПИН-код по Вашей карте установлен. Е

 {}
1: Добрый день , я забыл пин код дебетовой карты. Что мне делать?
2: Уточните, пожалуйста, Ваши ФИО и дату рождения.
1: Устинов Никита Андреевич 11.11.1111
2: Можно обратиться в банк по телефону 1 111 111-11-11 (звонок по России бесплатный, в том числе с мобильных телефонов) или я могу передать коллегам просьбу самостоятельно перезвонить Вам. Какой вариант удобнее для Вас?
 {'ТИП_КАРТЫ': {'start_pos': 33, 'end_pos': 43, 'text': 'дебетовой'}}
1: Здравствуйте, в каком банкомате можно снять деньги без комиссии
2: Согласно условиям Вашего тарифа, за каждую операцию снятия средств в банкомате предусмотрена комиссия - 1,1% от суммы снятия плюс 111 руб.
1: А 111 руб за что?
2: Это часть комиссии.
 {'ВАЛЮТА': {'start_pos': 204, 'end_pos': 209, 'text': 'руб.'}}
1: Я забыл пинкод карточки
2: Я могу передать коллегам, чтобы они связались с Вами или Вы можете самостоятельно позвонить в банк для получения нового ПИН-кода. Как Вам будет удобнее?
 {}
1: добрый день. подскажите пжста где в интерне

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [22]:
len(data)

6500

In [23]:
w2v = np.load('word2vec/w2v_model_tfidf_size300_window5_mc2.w2v.syn0.npy')


In [24]:
model = Word2Vec.load('word2vec/w2v_model_tfidf_size300_window5_mc2.w2v')

In [54]:
model.wv.most_similar_cosmul(positive=['тинькофф'])

[('тинькоф', 0.9431155323982239),
 ('тиньковы', 0.9234307408332825),
 ('тинкофф', 0.8928309679031372),
 ('тинкоф', 0.8490175604820251),
 ('ткс', 0.8455491065979004),
 ('тенькофф', 0.8355593681335449),
 ('тенёк', 0.8258935809135437),
 ('тинькова', 0.820256769657135),
 ('тенькоф', 0.8202194571495056),
 ('тикофф', 0.7970777750015259)]

In [26]:
print(data)

[ '1: Добрый день! Скажите, какие условия на снятие наличных в банкоматах ща границей? Карта Black\n2: Снятие в любых банкоматах без комиссии при выполнении условий тарифного плана. Однако, некоторые банки могут взимать собственную комиссия за свои услуги. Об этом информация отражается непосредственно перед снятием.\n1: В условиях тарифа я нашел информацию только по тарифам в рублях.\n1: Какая сумма эквивалентна 1111 р в, предполодим, долларах сша?\n1: по курсу цб рф? или это 111$, как было раньше?\n2: Сумма должна быть больше 1111 руб. с учетом конвертации. Ориентироваться нужно на курс Тинькофф Банка.\n2: https://www.tinkoff.ru/about/documents/exchange/\n2: "Для дебетовых карт" далее "С использованием ее реквизитов".\n'
 '1: Добрый день . Хотел бы уточнить у Вас , я могу снять деньги наличными со своей дебетовой карты в банкомате сбербанка без комисси ?\n2: Вы можете снимать наличные без комиссии в любом банкомате. Для этого Вам нужно: - снимать не менее 1 111 руб. за раз; - не превы

In [27]:
ans[1]

{'ВАЛЮТА': {'end_pos': 287, 'start_pos': 284, 'text': 'руб'},
 'НАЗВАНИЕ_БАНКА': {'end_pos': 119, 'start_pos': 110, 'text': 'сбербанка'},
 'СУММА_СНЯТИЯ': {'end_pos': 885, 'start_pos': 882, 'text': '111'},
 'ТИП_КАРТЫ': {'end_pos': 91, 'start_pos': 82, 'text': 'дебетовой'}}

In [28]:
ans[1]['ВАЛЮТА']['text']

'руб'

In [37]:
wordList[0]

['1',
 'добрый',
 'день',
 'скажите',
 'какие',
 'условия',
 'на',
 'снятие',
 'наличных',
 'в',
 'банкоматах',
 'ща',
 'границей',
 'карта',
 'black',
 '2',
 'снятие',
 'в',
 'любых',
 'банкоматах',
 'без',
 'комиссии',
 'при',
 'выполнении',
 'условий',
 'тарифного',
 'плана',
 'однако',
 'некоторые',
 'банки',
 'могут',
 'взимать',
 'собственную',
 'комиссия',
 'за',
 'свои',
 'услуги',
 'об',
 'этом',
 'информация',
 'отражается',
 'непосредственно',
 'перед',
 'снятием',
 '1',
 'в',
 'условиях',
 'тарифа',
 'я',
 'нашел',
 'информацию',
 'только',
 'по',
 'тарифам',
 'в',
 'рублях',
 '1',
 'какая',
 'сумма',
 'эквивалентна',
 '1111',
 'р',
 'в',
 'предполодим',
 'долларах',
 'сша',
 '1',
 'по',
 'курсу',
 'цб',
 'рф',
 'или',
 'это',
 '111',
 'как',
 'было',
 'раньше',
 '2',
 'сумма',
 'должна',
 'быть',
 'больше',
 '1111',
 'руб',
 'с',
 'учетом',
 'конвертации',
 'ориентироваться',
 'нужно',
 'на',
 'курс',
 'тинькофф',
 'банка',
 '2',
 'https',
 'www',
 'tinkoff',
 'ru',
 'abou